В данной лабораторной работе будут показаны основы анализа текстовой информации. В ходе её выполнения мы познакомимся с этапами предварительной подготовки данных, а также применим машинное обучение для задачи классификации.

Будем использовать датасет с текстами песен разных жанров: https://www.kaggle.com/mehedihasan9021/movie-script-dataset


> Задание выполняется в google colab. Чтобы редактировать ноутбук, не забудьте сохранить его копию на диске. Скачайте файл по ссылке, загрузите в сессионное хранилище и запускайте ячейки с кодом, следуя инструкциям.

Читаем набор данных

In [ ]:
import pandas as pd
data = pd.read_csv("dataset.csv")

In [ ]:
data.head()

,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genre     558 non-null    object
 1   lyrics    558 non-null    object
 2   SongInfo  558 non-null    object
dtypes: object(3)
memory usage: 13.2+ KB


In [ ]:
columns = data[['genre', 'lyrics']]

In [ ]:
columns

,genre,lyrics
0,Christian,"Who am I, that the Lord of all the earth Woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...
3,Christian,I can only imagine what it will be like When ...
4,Christian,I am not skilled to understand What God has w...
...,...,...
553,R&B,"Ha I dont care ha, about your past I just wan..."
554,R&B,Hoverin by my suitcase Tryin to find a warm ...
555,R&B,I dont know why I love you like I do After a...
556,R&B,"C. C. Rider Elvis Presley Well now see., C. ..."


Посмотрим какие жанры присутствуют в датасете. Для этого выведем уникальные значения столбца:

In [ ]:
columns['genre'].unique()

array(['Christian', 'Country', 'Hip-Hop', 'Pop', 'Rock', 'R&B'],
      dtype=object)

Посчитаем также сколько песен каждого жанра представлено:

In [ ]:
columns['genre'].value_counts()

Pop          100
Rock          95
Christian     94
Hip-Hop       91
R&B           91
Country       87
Name: genre, dtype: int64

Оставляем только 2 жанра, которые планируем отличать друг от друга. Для примера будут использованы Christian и Hip-Hop, позднее в формулировке задания нужно будет сгенерировать собственную пару жанров. 

In [ ]:
columns = columns[(columns.genre == 'Christian') | (columns.genre == 'Hip-Hop')]

###Предварительная обработка текстовых данных

Для того, чтобы применять текстовые данные для обучения той или иной модели машинного обучения, их нужно привести в подходящий для этого вид. В этом нам помогут следующие шаги предварительной подготовки:

####Приведение к нижнему регистру

Так как слова (например) "Beer" и "beer" будут считаться разными словами из-за разницы в регистре буквы b, а первая буква предложения, как правило, является заглавной, важно привести все слова предложения к нижнему регистру. Таким образом, "beer", встретившееся в начале предложения и в его середине, будут распознаны как одно слово.

In [ ]:
lowered = columns['lyrics'].str.lower()

In [ ]:
columns['lowered'] = lowered

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


####Токенизация

Имеющиеся тексты нужно разбить на отдельные слова. Такой процесс называется ***токенизация***.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)

In [ ]:
columns['tokened'] = tokened

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


####Удаление стоп-слов

***Стоп-словами*** называются распространённые слова, не несущие особой смысловой нагрузки, а значит никак не помогающие в последующей задаче классификации. Такие слова мы считаем шумом, и, соответственно, удаляем.

Существует специальный корпус стоп-слов на разных языках. Так как сейчас мы имеем дело с текстами на английском языке, выведем стоп-слова для английского:

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
noise = stopwords.words('english')

In [ ]:
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])

In [ ]:
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

In [ ]:
columns['without_stop'] = without_stop

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


####Лемматизация

У одного и того же слова бывают разные формы. Например, dances - форма слова dance. Лемматизация - это приведение к ***лемме*** - исходной форме слова. Её также необходимо применить к нашим данным, чтобы разные формы слова не считались за отдельные слова.

Существует целый ряд лемматизаторов, которые показывают себя с разной эффективностью на разных языках. Для работы с английским языком эффективен **WordNetLemmatizer**

In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
print(lemmatizer.lemmatize("dances"))

dance


In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

In [ ]:
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

In [ ]:
columns['lemmatized'] = lemma

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Таким образом, на каждом этапе мы добавляли столбец с той или иной модификацией, чтобы получить пригодные для обучения данные. Посмотрим на датафрейм, который получился:

In [ ]:
columns

,genre,lyrics,lowered,tokened,without_stop,lemmatized
0,Christian,"Who am I, that the Lord of all the earth Woul...","who am i, that the lord of all the earth woul...","[who, am, i, ,, that, the, lord, of, all, the,...",",, lord, earth, would, care, know, name, would...",",, lord, earth, would, care, know, name, would..."
1,Christian,Glory Revealed By His Wounds He was pierced ...,glory revealed by his wounds he was pierced ...,"[glory, revealed, by, his, wounds, he, was, pi...","glory, revealed, wounds, pierced, transgressio...","glory, revealed, wounds, pierced, transgressio..."
2,Christian,Lord of heaven and earth Lord of all creation...,lord of heaven and earth lord of all creation...,"[lord, of, heaven, and, earth, lord, of, all, ...","lord, heaven, earth, lord, creation, lord, hea...","lord, heaven, earth, lord, creation, lord, hea..."
3,Christian,I can only imagine what it will be like When ...,i can only imagine what it will be like when ...,"[i, can, only, imagine, what, it, will, be, li...","imagine, like, walk, side, imagine, eyes, see,...","imagine, like, walk, side, imagine, eyes, see,..."
4,Christian,I am not skilled to understand What God has w...,i am not skilled to understand what god has w...,"[i, am, not, skilled, to, understand, what, go...","skilled, understand, god, willed, ,, god, plan...","skilled, understand, god, willed, ,, god, plan..."
...,...,...,...,...,...,...
267,Hip-Hop,"Dirty dog Im, Im a dirty dog Im a dirty dog I...","dirty dog im, im a dirty dog im a dirty dog i...","[dirty, dog, im, ,, im, a, dirty, dog, im, a, ...","dirty, dog, im, ,, im, dirty, dog, im, dirty, ...","dirty, dog, im, ,, im, dirty, dog, im, dirty, ..."
268,Hip-Hop,"Regulators, we regulate any stealing of his p...","regulators, we regulate any stealing of his p...","[regulators, ,, we, regulate, any, stealing, o...","regulators, ,, regulate, stealing, property, d...","regulators, ,, regulate, stealing, property, d..."
269,Hip-Hop,Have you ever met a girl that you tried to da...,have you ever met a girl that you tried to da...,"[have, you, ever, met, a, girl, that, you, tri...","ever, met, girl, tried, date, year, make, love...","ever, met, girl, tried, date, year, make, love..."
270,Hip-Hop,"Yo, yo, yo They wanna know Whos that girl? (...","yo, yo, yo they wanna know whos that girl? (...","[yo, ,, yo, ,, yo, they, wan, na, know, whos, ...","yo, ,, yo, ,, yo, wan, na, know, whos, girl, ?...","yo, ,, yo, ,, yo, wan, na, know, whos, girl, ?..."


####Разделим данные на обучающую и тестовую выборки

x_train - тексты, на которых мы обучаем модель. В данном случае мы используем столбец lemmatized, так как он содержит данные, прошедшие все этапы подготовки 

> 



y_train - жанры, соответствующие текстам, на которых модель обучается - столбец genre


> 


x_test - точно такие же тексты из набора данных, на которых мы будем проверять, насколько модель научилась предсказывать жанр


> 



y_test - жанры, соответствующие x_test. т.е. мы смотрим, насколько предсказания соответствуют содержимому этой переменной, чтобы оценить качество обучения

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)

In [ ]:
columns.genre.value_counts()

Christian    94
Hip-Hop      91
Name: genre, dtype: int64

####Векторизация

Следующий этап - ***векторизация***, то есть представление текста в численном виде, чтобы закодированные данные в дальнейшем использовать на модели.

Иногда помимо отдельных слов для улучшения качества обучения будет полезно использовать также комбинации из 2-3 слов. Здесь мы используем векторизатор CountVectorizer, который имеет встроенный метод - мешок n-грамм.

> n = 1 - униграмма (для обучения используются слова по отдельности)

> n = 2 - биграмма (для обучения используются пары слов)

> n = 3 - триграмма (для обучения используются тройки слов)







In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Задаём для векторизатора ngram_range=(1, 3), то есть используем все варианты n от 1 до 3 и прибегаем как к униграммам, так и к биграммам и триграммам:

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

### Классификация

Для задачи классификации используем Наивный Байесовский Классификатор - простой вероятностный классификатор, основанный на применении теоремы Байеса со строгими предположениями о независимости. 

In [ ]:
# импортируем байесовский классификатор
from sklearn.naive_bayes import MultinomialNB 

In [ ]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB()

In [ ]:
# тестовую выборку просто векторизировали
vectorized_x_test = vectorizer.transform(x_test)

Посмотрим предсказания для тестовой выборки

In [ ]:
clf.predict(vectorized_x_test)

array(['Christian', 'Christian', 'Hip-Hop', 'Christian', 'Christian',
       'Hip-Hop', 'Christian', 'Christian', 'Hip-Hop', 'Hip-Hop',
       'Christian', 'Hip-Hop', 'Christian', 'Hip-Hop', 'Christian',
       'Christian', 'Christian', 'Christian', 'Hip-Hop', 'Christian',
       'Christian', 'Hip-Hop', 'Christian', 'Hip-Hop', 'Hip-Hop',
       'Hip-Hop', 'Hip-Hop', 'Christian', 'Hip-Hop', 'Christian',
       'Christian', 'Christian', 'Hip-Hop', 'Hip-Hop', 'Hip-Hop',
       'Christian', 'Christian', 'Hip-Hop', 'Christian', 'Hip-Hop',
       'Hip-Hop', 'Hip-Hop', 'Christian', 'Hip-Hop', 'Christian',
       'Hip-Hop', 'Hip-Hop', 'Hip-Hop', 'Hip-Hop', 'Christian', 'Hip-Hop',
       'Christian', 'Hip-Hop', 'Christian', 'Hip-Hop', 'Hip-Hop'],
      dtype='<U9')

Получим оценки классификации:

In [ ]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   Christian       0.96      0.96      0.96        27
     Hip-Hop       0.97      0.97      0.97        29

    accuracy                           0.96        56
   macro avg       0.96      0.96      0.96        56
weighted avg       0.96      0.96      0.96        56



In [ ]:
!jupyter nbconvert --to html /content/Lab2_TextAnalysis.ipynb

[NbConvertApp] Converting notebook /content/Lab2_TextAnalysis.ipynb to html
[NbConvertApp] Writing 401710 bytes to /content/Lab2_TextAnalysis.html


### Задание

#### Задание 1

Запустить ячейку ниже, чтобы получить 2 жанра. Для полученных жанров провести все этапы предварительной обработки текста (как в примере), обучить наивный байесовский классификатор, численно оценить его работу.

In [ ]:
import random 
lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B'] 
print('ваши жанры', random.choice(lst), 'и', random.choice(lst)) 

ваши жанры Country и Rock


In [ ]:
songs = pd.read_csv("dataset.csv")
filtered_songs = songs[['genre', 'lyrics']][(songs.genre == 'Country') | (songs.genre == 'Rock')]

In [ ]:
lowered = filtered_songs['lyrics'].str.lower()
filtered_songs['lowered'] = lowered

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

In [ ]:
tokened = filtered_songs.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
filtered_songs['tokened'] = tokened

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
noise = stopwords.words('english')
noiseless = filtered_songs['tokened'].apply(lambda x: [item for item in x if item not in noise])
noiseless_arr = []
for iter in noiseless:
    noiseless_arr.append(", ". join(iter))
filtered_songs['noiseless'] = noiseless_arr

In [ ]:
import nltk 
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
lemmatized = filtered_songs['noiseless'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemmatized_arr = []
for iter in lemmatized: 
  lemmatized_arr.append(", ".join(iter))
filtered_songs['lemmatized'] = lemmatized_arr

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(filtered_songs.lemmatized, filtered_songs.genre, train_size = 0.3)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,3))
vectorized_x_train = vectorizer.fit_transform(x_train)

In [ ]:
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

In [ ]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

     Country       0.64      0.61      0.62        57
        Rock       0.70      0.72      0.71        71

    accuracy                           0.67       128
   macro avg       0.67      0.67      0.67       128
weighted avg       0.67      0.67      0.67       128



#### Задание 2

Найти (нагуглить) по песне каждого из жанров, которые Вам достались, после необходимой обработки их текстов определить жанр обеих песен с помощью обученной в ходе выполнения предыдущего пункта модели.

In [ ]:
songs2 = pd.read_csv('mydataset.csv')
columns = songs2[['lyrics']]

In [ ]:
lowered = columns['lyrics'].str.lower()
columns['lowered'] = lowered

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)

In [ ]:
columns['tokened'] = tokened

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

columns['without_stop'] = without_stop

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

columns['lemmatized'] = lemma

In [ ]:
x_test = columns['lemmatized']
vectorized_x_test = vectorizer.transform(columns['lemmatized'])
pred = clf.predict(vectorized_x_test)

In [ ]:
pred

array(['Country', 'Rock'], dtype='<U7')

Country: Country Roads, Rock: Rock You Like A Hurricane -> Success

#### Задание 3

С помощью набора данных по ссылке аналогичным образом научить модель отличать тексты песен Дэвида Боуи от текстов песен Пола МакКартни 
https://www.kaggle.com/italomarcelo/dataset-lyrics-music-mini

In [ ]:
songs3 = pd.read_csv("dataset-lyrics-musics-mini.csv")
filt_songs = songs3[["letra","cantorNome"]][(songs3.cantorNome=="david-bowie") | (songs3.cantorNome=="paul-mccartney")]
filt_songs

,letra,cantorNome
0,"I, I will be king. And you, you will be queen....",david-bowie
1,"Didn't know what time it was,. The lights were...",david-bowie
2,Ground control to Major Tom. Ground control to...,david-bowie
3,It's a god-awful small affair. To the girl wit...,david-bowie
4,I know when to go out. And when to stay in. Ge...,david-bowie
...,...,...
942,He's just a young boy looking for a way to fin...,paul-mccartney
943,How can I hope to reach your love. Help me to ...,paul-mccartney
944,I like it. Please don't take my heart away. It...,paul-mccartney
945,Yvonne is the one I´ve been counting on. She s...,paul-mccartney


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
lowered = filt_songs['letra'].str.lower()
filt_songs['lowered'] = lowered
nltk.download('punkt')
tokened = filt_songs.apply(lambda row: nltk.word_tokenize(row['letra']), axis=1)
filt_songs['tokened'] = tokened

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
noise = stopwords.words('english')
withoutstop = filt_songs['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
filt_songs['without_stop'] = without_stop
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmatized = filt_songs['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
filt_songs['lemmatized'] = lemma

In [ ]:
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmatized = filt_songs['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
filt_songs['lemmatized'] = lemma

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(filt_songs.lemmatized, filt_songs.cantorNome, train_size = 0.7)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 2))
vectorized_x_train = vectorizer.fit_transform(x_train)
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

In [ ]:
from sklearn.metrics import *

In [ ]:
print(classification_report(y_test, pred))

                precision    recall  f1-score   support

   david-bowie       0.78      0.76      0.77       148
paul-mccartney       0.75      0.77      0.76       137

      accuracy                           0.76       285
     macro avg       0.76      0.76      0.76       285
  weighted avg       0.77      0.76      0.76       285

